# Download a slice of the ERA5 dataset

This notebooks downloads a large slice of atmospheric temperature at 500 hPa from the [ERA5 dataset](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5).
You will need this data if you want to run the demo notebooks.
The slice corresponds to a time period between 2016-01-01 and 2020-02-29, with a time delta of 4 hours.
The size of the slice is approximately 38 GB, and will be stored on disk as a Zarr store.

Import some utility functions from the `utils.py` module.

In [1]:
import xarray as xr
from utils import (
    display_size,
    generate_time_ranges,
    resample_array,
    write_to_zarr,
)

Open the ERA5 zarr file on the cloud, chunking along the time dimension.

In [2]:
era5_path = "gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2"
era5 = xr.open_zarr(era5_path, chunks={"time": "auto"})

Extract the temperature variable, and select pressure level 500 and a time slice between 1 Jan 2016 and 29 Feb 2020.

In [3]:
t500 = era5["temperature"].sel(level=500, time=slice("2016-01-01", "2020-02-29"))

In [4]:
t500

<xarray.DataArray 'temperature' (time: 36504, latitude: 721, longitude: 1440)> Size: 152GB
dask.array<getitem, shape=(36504, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 292kB 2016-01-01 ... 2020-02-29T23:00:00
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    level      int64 8B 500
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [5]:
display_size(t500)

Array size: 141.19 GiB


We will downsample the temporal resolution to 4 hours, to reduce the file size. This sampling frequency is still high enough to capture all scales of interest.

In [6]:
t500 = resample_array(t500, hours=4)

In [7]:
t500

<xarray.DataArray 'temperature' (time: 9126, latitude: 721, longitude: 1440)> Size: 38GB
dask.array<getitem, shape=(9126, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 73kB 2016-01-01 ... 2020-02-29T20:00:00
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    level      int64 8B 500
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [8]:
display_size(t500)

Array size: 35.30 GiB


In [8]:
t500.time

<xarray.DataArray 'time' (time: 9126)> Size: 73kB
array(['2016-01-01T00:00:00.000000000', '2016-01-01T04:00:00.000000000',
       '2016-01-01T08:00:00.000000000', ..., '2020-02-29T12:00:00.000000000',
       '2020-02-29T16:00:00.000000000', '2020-02-29T20:00:00.000000000'],
      shape=(9126,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 73kB 2016-01-01 ... 2020-02-29T20:00:00
    level    int64 8B 500

Now we will save the downsampled ERA5 slice to disk as a Zarr store, which will later enable lazy, chunk-wise loading when we work on it with SVD-ROM. Because the slice we want to download is quite large, it is best to download it in separate stages. The code bellow splits the slice into different time ranges and downloads each one separately, appending them along the time dimension. Because the data is chunked along the time dimension, we will download the data time step by time step. Note that the download will take a while!

In [9]:
time_periods = generate_time_ranges("2016-01-01", "2020-02-29", num_periods=7)
t500_list = []
for period in time_periods:
    t500_slice = t500.sel(time=slice(period[0], period[1]))
    t500_list.append(t500_slice)

In [10]:
t500_list[0]

<xarray.DataArray 'temperature' (time: 1308, latitude: 721, longitude: 1440)> Size: 5GB
dask.array<getitem, shape=(1308, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 10kB 2016-01-01 ... 2016-08-05T20:00:00
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    level      int64 8B 500
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

As shown below, each of the splits has a size of approximately 5 GB.

In [11]:
for da in t500_list:
    print(
        f"Start: {da.time.values[0].astype('datetime64[h]')}, "
         f"End: {da.time.values[-1].astype('datetime64[h]')}, "
         f"Size: {da.nbytes/(1024**3):.2f} GiB"
    )

Start: 2016-01-01T00, End: 2016-08-05T20, Size: 5.06 GiB
Start: 2016-08-06T00, End: 2017-03-11T20, Size: 5.06 GiB
Start: 2017-03-12T00, End: 2017-10-15T20, Size: 5.06 GiB
Start: 2017-10-16T00, End: 2018-05-21T20, Size: 5.06 GiB
Start: 2018-05-22T00, End: 2018-12-25T20, Size: 5.06 GiB
Start: 2018-12-26T00, End: 2019-07-31T20, Size: 5.06 GiB
Start: 2019-08-01T00, End: 2020-02-29T20, Size: 4.94 GiB


In [ ]:
filename = "era5_slice.zarr"

Now download separately each array in the list. Note that they are all saved to the same zarr store, appending along the time dimension. Because we have chunked the ERA5 slice along the time dimension, the data will be downloaded, compressed and saved to disk time step by time step. By default the data will be saved to the `data/` subfolder. Warning: the download will take a while!

In [ ]:
write_to_zarr(t500_list[0], filename)

In [ ]:
write_to_zarr(t500_list[1], filename)

In [ ]:
write_to_zarr(t500_list[2], filename)

In [ ]:
write_to_zarr(t500_list[3], filename)

In [ ]:
write_to_zarr(t500_list[4], filename)

In [ ]:
write_to_zarr(t500_list[5], filename)

In [ ]:
write_to_zarr(t500_list[6], filename)